In [ ]:
!source activate mindsrq
import moxing as mox
import cv2
import numpy as np
from PIL import Image
import mindspore.dataset.vision.py_transforms as py_trans
import mindspore
print(mindspore.__version__)

In [ ]:
root_path = 'obs://mindspore-srq/dataset/archive/lfw-deepfunneled/lfw/'
file_list = mox.file.list_directory(root_path)

imgs_path =  root_path+file_list[0]+'/AJ_Cook_0001.jpg'

fc = mox.file.read( imgs_path, binary=True )
fileNPArray=np.frombuffer(fc,np.uint8)

x = cv2.imdecode( fileNPArray, cv2.IMREAD_COLOR )

image = Image.fromarray(cv2.cvtColor(x,cv2.COLOR_BGR2RGB))  
# image.show()
image = c_vision.Invert(image)
display(image)

print(x.shape)

In [1]:
import mindspore
mindspore.context.set_context(device_target="Ascend")
mindspore.context.set_context(device_id=0)
import mindspore.dataset as ds
from dataset import LFWImagePairList
import mindspore.dataset.vision.py_transforms as py_trans
from mindspore.dataset.transforms.py_transforms import Compose
from get_models import SpherefaceModel_mind, CosfaceModel_mind, ArcfaceModel_mind
from utils import cal_distance

import numpy as np
m = ArcfaceModel_mind()

dataset_generator = LFWImagePairList(out_shape=(112,112))
print(len(dataset_generator))
dataset = ds.GeneratorDataset(dataset_generator, column_names=['img1', 'img1_', 'img2', 'img2_', 'sameflag'],num_parallel_workers=12 , shuffle=False)

transforms_list = [py_trans.ToTensor(),py_trans.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
dataset = dataset.map(operations=Compose(transforms_list), input_columns=['img1'])
dataset = dataset.map(operations=Compose(transforms_list), input_columns=['img1_'])
dataset = dataset.map(operations=Compose(transforms_list), input_columns=['img2'])
dataset = dataset.map(operations=Compose(transforms_list), input_columns=['img2_'])
dataset = dataset.batch(batch_size=50)

cast = mindspore.ops.Cast()

temp = dataset.create_dict_iterator()
sqrt = mindspore.ops.Sqrt()


cc_num = 0
all_num = 0
# threshold = 0.2245
threshold=0.525
for data in temp:
    

    # data = next(temp)
    # input = mindspore.Tensor( np.random.randn(1,3,112,96) , dtype=mindspore.float32)
    img1 = cast( data['img1'],  mindspore.float32)
    img1_ = cast( data['img1_'],  mindspore.float32)
    img2 = cast( data['img2'],  mindspore.float32)
    img2_ = cast( data['img2_'],  mindspore.float32)
   
    f1 = m.forward( img1, img1_ )
    f2 = m.forward(img2, img2_)

    distance = cal_distance( f1.asnumpy(),f2.asnumpy() )  
    pred = np.zeros(distance.shape[0], dtype=np.int8)
    print(distance[0])
    pred[ distance>threshold ] = 1
    
    cc_num = cc_num + np.sum(pred== np.array(data['sameflag'].asnumpy(), dtype=np.int8))
    all_num = all_num + distance.shape[0]
    # print(cc_num,all_num)
print( cc_num/len(dataset_generator) )

[WARNING] ME(14321:281473523128624,MainProcess):2021-08-13-12:52:53.735.499 [mindspore/_check_version.py:207] MindSpore version 1.1.1 and "te" wheel package version 1.0 does not match, reference to the match info on: https://www.mindspore.cn/install
MindSpore version 1.1.1 and "topi" wheel package version 0.6.0 does not match, reference to the match info on: https://www.mindspore.cn/install
[WARNING] ME(14321:281473523128624,MainProcess):2021-08-13-12:52:54.267.129 [mindspore/ops/operations/array_ops.py:2302] WARN_DEPRECATED: The usage of Pack is deprecated. Please use Stack.


INFO:root:Using MoXing-v1.17.3-d858ff4a
INFO:root:Using OBS-Python-SDK-3.20.9.1


Execute the process of loading checkpoint files.
Loading checkpoint files process is finished.
6000
0.58692026
0.45469654
0.7714607
0.4858347
0.708799
0.6084898
0.58672464
0.29236358
0.3483346
0.29094344
0.531161
0.5694102
0.6908098
0.5745831
0.62233204
0.64995587
0.67952734
0.5291283
0.4921826
0.22836705
0.5750542
0.54408765
0.37494993
0.38003474
0.69782436
0.77282906
0.82658464
0.459174
0.7969748
0.6007374
0.5055694
0.21173541
0.47692654
0.47331268
0.3561453
0.5977786
0.62208486
0.7720188
0.7720637
0.59974074
0.6668122
0.50313276
0.22981286
0.14224643
0.33070046
0.29068562
0.31124225
0.56772816
0.7304874
0.7778332
0.73667926
0.4392547
0.7073376
0.5669897
0.27846962
0.36163184
0.42752796
0.29895884
0.45111492
0.33346492
0.4894564
0.64905447
0.84824896
0.5610016
0.6047585
0.8208344
0.46744934
0.45034394
0.32916695
0.49273196
0.16428682
0.37667447
0.37922958
0.74554926
0.6537675
0.6789131
0.45324627
0.6627202
0.4061658
0.19570032
0.39752072
0.46371433
0.39493713
0.5388031
0.7152582
0.68

In [ ]:
from get_models import SpherefaceModel_mind
import numpy as np
import mindspore 

mindspore.context.set_context(device_target="Ascend")

m = SpherefaceModel_mind()

x = mindspore.Tensor( np.random.randn(1,3,96,112), mindspore.float32 )

# print(x.shape)

y = m.forward( x , x)
print(y.shape)